- Usaremos diversas bibliotecas: nltk, pandas e scikit-learn
- Lembre se que algumas bibliotecas devem ser instaladas!
- Usei "pip install *nome da biblioteca*" pra fazer a instalação

In [1]:
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# Ler o arquivo com os dados, mostre uma amostra do arquivo e exibe a 
# contagem de cada uma das colunas.

In [36]:
dataset = pd.read_csv('~/Documents/workspace/gsi024/tp4/reforma_previdencia_rotulado.csv',encoding='utf-8', sep=';')
# dataset = pd.read_csv(r"C:\Users\Miani\Dropbox\Rodrigo\Trabalho\UFU\Disciplinas\Organização e Recuperação da Informação\Exercícios\TP\TP4\Tweets_Mg.csv",encoding='utf-8')

In [37]:
dataset.head()

,Tweet,Classificação
0,Pressionem Queremos a reforma da previdência d...,Positivo
1,#EPTV1 Parabéns pela imparcialidade. Mostrando...,Positivo
2,@dep_paulinho TU É UM CANALHA. INDO CONTRA A R...,Positivo
3,"Em mais um golpe do Centrão, aquela grande fre...",Positivo
4,Eles acham que o objetivo é só lula livre e ca...,Positivo


In [38]:
dataset.count()

Tweet            2232
Classificação    2232
dtype: int64

In [6]:
# Vamos contar quantos tweets de cada tipo existem: neutro, positivo e negativo.

In [44]:
dataset[dataset.Classificação=='Neutro'].count()

Tweet            780
Classificação    780
dtype: int64

In [45]:
dataset[dataset.Classificação=='Positivo'].count()

Tweet            740
Classificação    740
dtype: int64

In [46]:
dataset[dataset.Classificação=='Negativo'].count()

Tweet            712
Classificação    712
dtype: int64

In [10]:
# Precisamos criar váriaveis diferentes para armazenar os tweets e a sua classificação

In [48]:
tweets = dataset['Tweet'].values
classes = dataset['Classificação'].values

In [12]:
# Vamos treinar o nosso primeiro modelo de classificação de texto. 
# Algumas coisas são importantes aqui:

# 1) Precisamos definir como representar os tweets (BoW)
# 2) Precisamos definir qual algoritmo de classificação será usado 


### Breve explicação sobre o funcionamento de algoritmos de classificação: ###
# O algoritmo irá relacionar o conteúdo da BoW (1,0,2,0,1) com a respectiva classificação 'Neutro'. Usando diferentes critérios e com auxílio dos dados
# analisados, o algoritmo irá criar "regras" para identificar/generalizar cada uma das classificações - Neutro, Positivo e Negativo. Chamamos esse conjunto de regras de "modelo".
# Feito isso, quando o modelo receber um novo tweet (BoW) sem a classificação, com base nas regras que foram criadas, ele irá tentar "adivinhar" qual será a classe daquele tweet.
# ####

In [13]:
# Para resolver o problema 1), vamos usar somente o TF - CountVectorizer. Essa função já limpa aqueles caracteres esquisitos que vimos lá em cima.

# Na linha 1, criamos um objeto do tipo CountVectorizer chamado vectorizer. Após isso, na linha 2, usamos o objeto vectorizer para calcular a frequência de todas as palavras da lista de tweets e armazenamos seu retorno em freq_tweets.

In [51]:
vectorizer = CountVectorizer(ngram_range=(1,2))
freq_tweets = vectorizer.fit_transform(tweets)

In [15]:
# Para resolver o problema 2), vamos trabalhar com um algoritmo de classificação chamado de Naive Bayes. Ele é baseado em probabilidades.
# Na linha 1, criamos um objeto chamado modelo do tipo Naive Bayes Multinomial.
# Na linha 2, treinamos o modelo usando a frequência de palavras (freq_tweets) e as classes de cada instância.

In [52]:
modelo = MultinomialNB()
modelo.fit(freq_tweets,classes)

MultinomialNB()

In [17]:
# Vamos fazer alguns testes "manuais". Ou seja, fornecer como entrada para o modelo alguns tweets
# e deixar que ele faça a classificação. Na opinião de vocês, qual seria a classificação para cada um desses
# tweets?

In [59]:
testes = ['A reforma da previdencia foi muito boa','A reforma foi péssima','Não gostei da reforma','A reforma foi tenebrosa','A reforma da previdência é uma das reformas já feitas']
print(testes)

['A reforma da previdencia foi muito boa', 'A reforma foi péssima', 'Não gostei da reforma', 'A reforma foi tenebrosa', 'A reforma da previdência é uma das reformas já feitas']


In [19]:
# Calculo a BoW dos tweets dentro da variável testes usando o TF.

In [60]:
freq_testes = vectorizer.transform(testes)

In [21]:
# Faço a classificação dos tweets de testes usando o modelo treinado.

In [61]:
modelo.predict(freq_testes)

array(['Neutro', 'Negativo', 'Positivo', 'Positivo', 'Positivo'],
      dtype='<U8')

In [23]:
# Vimos que o modelo funciona! Tem alguns erros mas isso faz parte do processo. 
# O próximo passo agora é fazer uma avaliação mais robusta do modelo. 
# Vamos usar uma parte da base de dados para treinar e a outra parte para testar. 
# Uma maneira de se fazer isso é usando um método chamado de "Cross Validation" ou "Validação cruzada".
# Essta técnica consiste em dividir todo o conjunto de dados em K partes, que serão chamadas de folds. 
# Dessas partes, uma será separada para teste e as outras restantes serão usadas para treinar o modelo.

## Exemplo ##

# Para k = 10 , imagine que todo nosso dado de treino foi dividido em 10 partes distintas.
# Assim, o modelo será treinado com 9 partes, e testado com a parte restante. Esse processo é repetido até que o modelo seja treinado e testado com todas as partes do dado.

# A variável "resultados" guarda as previsões feitas pelo pelo modelo usando a validação cruzada.

In [62]:
resultados = cross_val_predict(modelo, freq_tweets, classes, cv=10)

In [25]:
# Pronto! Modelo treinado e validado! Como descobrir o desempenho do modelo? Inicialmente, usaremos uma
# medida chamada de Acurácia que nada mais é do que o percentual de acertos que o modelo teve.

In [64]:
metrics.accuracy_score(classes,resultados)

0.6097670250896058

In [27]:
# E se eu quiser saber o desempenho por cada uma das classes? Talvez o modelo acerte mais uma classe
# do que a outra...

In [65]:
print(metrics.classification_report(classes,resultados))

              precision    recall  f1-score   support

    Negativo       0.59      0.58      0.58       712
      Neutro       0.68      0.56      0.62       780
    Positivo       0.58      0.69      0.63       740

    accuracy                           0.61      2232
   macro avg       0.61      0.61      0.61      2232
weighted avg       0.62      0.61      0.61      2232



In [29]:
# E seu eu quiser saber a quantidade de acertos por classe? Nesse caso precisamos mostrar 
# a matriz de confusão.

In [66]:
print (pd.crosstab(classes, resultados, rownames=['Real'], colnames=['Predito'], margins=True))

Predito   Negativo  Neutro  Positivo   All
Real                                      
Negativo       414     116       182   712
Neutro         150     439       191   780
Positivo       141      91       508   740
All            705     646       881  2232
